In [70]:
from pathlib import Path
import h5py
import datetime
import json
import os
from pathlib import Path

In [71]:
def create_group_to_fill(TYPE, where, name):
    group=where.create_group(name)
    group.attrs["NX_class"]=TYPE
    return group

In [72]:
def write_data(dati, where):
    for row in dati:
        if (isinstance(dati[row], str) and row != "m_def") or isinstance(dati[row], (int, float, bool)):
            where.create_dataset(row, data=dati[row])
        elif isinstance(dati[row], dict):
            if dati[row].keys() == {"value", "unit", "direction"}:
                where.create_dataset(row, data=dati[row]["value"])
                where[row].attrs["units"]=dati[row]["unit"]
                where[row].attrs["direction"] = dati[row]["direction"]
            elif dati[row].keys() <= {"value", "unit"}:
                where.create_dataset(row, data=dati[row]["value"])
                where[row].attrs["units"]=dati[row]["unit"]
            elif all(isinstance(x, str) for x in dati[row].values()) and "m_def" not in dati[row].keys():
                values= list(dati[row].values())
                keys=list(dati[row].keys())
                print(values, keys)
                nome_dataset = keys[0]
                valore_dataset = values[0]
                where.create_dataset(nome_dataset, data= valore_dataset)
                for attr, inst in zip(keys[1:], values[1:]):
                    where[nome_dataset].attrs[attr]=inst
            elif "m_def" in dati[row].keys():
                newwhere=create_group_to_fill(dati[row]["m_def"], where, row)
                write_data(dati[row], newwhere)

In [73]:
mio_dict={"a": "aa", "b": "bb"}

list(mio_dict.values())

['aa', 'bb']

In [74]:
def write_from_json(json_input, where):
    with open(json_input, "r", encoding="utf-8") as file:
        dati = json.load(file)
        write_data(dati, where)

In [75]:
def aprire_jsons(directory, entry):
    for file in os.listdir(directory):
        if Path(file).suffix == ".json" and "entry" in file:
            write_from_json(os.path.join(directory, file), entry)

    for file in os.listdir(directory):
        if Path(file).suffix == ".json" and "sample" in file:
            sample=create_group_to_fill("NXsample", entry, "sample")
            write_from_json(os.path.join(directory, file), sample)
        if Path(file).suffix == ".json" and "user" in file:
            user=create_group_to_fill("NXuser", entry, "user")
            write_from_json(os.path.join(directory, file), user)
        if Path(file).suffix == ".json" and "instrument" in file:
            instr=create_group_to_fill("NXinstrument", entry, "instrument")
            write_from_json(os.path.join(directory, file), instr)
        if Path(file).suffix == ".json" and "measurement" in file:
            measure=create_group_to_fill( "NXem_measurement", entry, "measurement")
            write_from_json(os.path.join(directory, file), measure)
            

In [76]:
with h5py.File("afm_file_auto1.nxs", "w") as f:
    entry = f.create_group("entry")
    entry.attrs["NX_class"] = "NXentry"
    entry.create_dataset("definition", data="NXafm")
    entry["definition"].attrs["version"] = "v2024.02"
    entry["definition"].attrs["URL"] = "https://github.com/FAIRmat-NFDI/nexus_definitions/blob/fairmat/contributed_definitions/NXafm.nxdl.xml"
    aprire_jsons("./jsons_for_afm/" , entry)
    #### Completiamo generando i link necessari
    scan = entry["instrument"].create_group ("scan_environment")
    scan.attrs["NX_class"] = "NXenvironment"
    scan["height_piezo_sensor"] = entry["instrument"]["height_piezo_sensor"]
    scan["XY_piezo_sensor"] = entry["instrument"]["XY_piezo_sensor"]
    scan["tip_temp_sensor"] = entry["instrument"]["tip_temp_sensor"]
    scan.create_dataset("tip_temp", data=25)
    scan["tip_temp"].attrs["units"] = "celsius"
    #### Reproducibility indicators
    reprind = entry.create_group("reproducibility_indicators")
    reprind.attrs["NX_class"] = "NXcollection"
    reprind["cantilever_oscillator"] = entry["instrument"]["cantilever"]["cantilever_oscillator"]
    reprind["cantilever_tip_temp"] = entry["instrument"]["scan_environment"]["tip_temp"]
    #### Resolution indicators
    resind = entry.create_group("resolution_indicators")
    resind.attrs["NX_class"] = "NXcollection"
    resind["oscillator_excitation"] = entry["instrument"]["cantilever"]["cantilever_oscillator"]["oscillator_excitation"]
    resind["cantilever_tip_temp"] = entry["instrument"]["scan_environment"]["tip_temp"]
    resind["amplitude_excitation"] = entry["instrument"]["cantilever"]["cantilever_oscillator"]["phase_lock_loop"]["amplitude_excitation"]

In [88]:
def extract_data_from_tif(tif_image):
    import tifffile as tf
    import numpy as np
    name = f"{tif_image}"
    array = tf.imread(tif_image)
    array = array.astype(np.float32)
    array /= array.max()
    tf.imwrite("copy.tif", array)
    return name, array

In [89]:
with h5py.File("sem_file_auto.nxs", "w") as f:
    entry = f.create_group("entry")
    entry.attrs["NX_class"] = "NXentry"
    entry.create_dataset("definition", data="NXem")
    entry["definition"].attrs["version"] = "v2024.02"
    entry["definition"].attrs["URL"] = "https://github.com/FAIRmat-NFDI/nexus_definitions/blob/a85e10cd0289f4e44b0fec011ff54703e6705383/contributed_definitions/NXem.nxdl.xml"
    aprire_jsons("./jsons_for_sem/", entry)
    for idx, tif_file in enumerate(sorted(Path("./tiffs_for_sem").glob("*.tif"))):
        img_name, img_data = extract_data_from_tif(tif_file)
        image = entry["measurement"]["event"].create_group(f"image_{idx+1}")
        image.attrs["NX_class"] = "NXimage"
        image_2d = image.create_group("image_2d")
        image_2d.attrs["NX_class"] = "NXdata"
        image_2d.create_dataset("title", data=img_name)
        image_2d.create_dataset("real", data=img_data)
        image_2d["real"].attrs["long_name"] = "SEM Image Intensity"
        image_2d.create_dataset("axis_i", data=np.arange(img_data.shape[0]))
        image_2d.create_dataset("axis_j", data=np.arange(img_data.shape[1]))
        image_2d.attrs["signal"] = "real"
        image_2d.attrs["axes"] = ["axis_i", "axis_j"]
        image_2d.attrs["axis_i_indices"] = 0
        image_2d.attrs["axis_j_indices"] = 1

############ Comunque grandi passi avanti mi manca di potere visualizzare l'immagine su nomad

['name of the program', 'version'] ['program', 'version']


In [90]:
from nexusformat.nexus import *

test = nxload("sem_file_auto.nxs")
print(test.tree)

root:NXroot
  entry:NXentry
    coordinate_system:NXcoordinate_system
      handedness = 'right_handed'
      origin = 'sample'
      type = 'cartesian'
    definition = 'NXem'
      @URL = 'https://github.com/FAIRmat-NFDI/nexus_definiti...'
      @version = 'v2024.02'
    end_time = ''
    experiment_alias = 'alias'
    experiment_description = 'short description'
    experiment_identifier = 'experiment_identifier'
    measurement:NXem_measurement
      event:NXem_event_data
        image:NX_image
          image_2d:NXdata
            title = ''
          instrument:NXem_instrument
          optics:NXem_optical_system
            dynamic_focus_correction = True
            field_of_view = 4
              @units = 'm'
            magnification = 0
            probe_current = 4
              @units = 'A'
            tilt_correction = True
            working_distance = 4
              @units = 'm'
        image_1:NXimage
          image_2d:NXdata
            @axes = ['axis_i', 'axis_j']

In [66]:
import tifffile as tf
import numpy as np
image = tf.imread('./tiffs_for_sem/U1502_metal_cross_5um_015.tif')
image = image.astype(np.float32)
image /= image.max()
print(image.shape, image.dtype)

tf.imwrite("copy.tif", image)

(1094, 1536) uint16


In [22]:
import tifffile
import json

def leggi_metadati_tif(percorso):
    metadati_pagine = []
    with tifffile.TiffFile(percorso) as tif:
        for i, page in enumerate(tif.pages):
            # Convertiamo i tag in dict leggibile
            tags = {}
            for tag in page.tags.values():
                try:
                    tags[tag.name] = str(tag.value)
                except Exception:
                    tags[tag.name] = "Non leggibile"
            
            metadati_pagine.append({
                "Pagina": i + 1,
                "Dimensioni immagine": page.shape,
                "Tipo dati": page.dtype.name,
                "Descrizione": page.description,
                "Tag": tags
            })
    
    return {
        "Nome file": percorso.split("/")[-1],
        "Pagine": metadati_pagine
    }

# Lista dei file TIFF
file_paths = [
    "./tiffs_for_sem/run2_area_01.tif",
    "./tiffs_for_sem/U1502_metal_cross_5um_015.tif"
]

# Creiamo il JSON usando il nome del file come chiave
metadati = {fp.split("/")[-1]: leggi_metadati_tif(fp) for fp in file_paths}

# Salvataggio in JSON
with open("metadati_tif.json", "w") as f:
    json.dump(metadati, f, indent=4, ensure_ascii=False)


In [13]:
import tifffile
import json

def leggi_metadati_tif(percorso):
    with tifffile.TiffFile(percorso) as tif:
        page = tif.pages[0]
        return {
            "Nome file": percorso.split("/")[-1],
            "Dimensioni immagine": page.shape,
            "Tipo dati": page.dtype.name,
            "Descrizione": page.description,
            "Tag": {tag.name: tag.value for tag in page.tags.values()}
        }

# Uso con più file
file_paths = [
    "./tiffs_for_sem/run2_area_01.tif",
    "./tiffs_for_sem/U1502_metal_cross_5um_015.tif"
]

metadati = {f"file{i+1}": leggi_metadati_tif(fp) for i, fp in enumerate(file_paths)}

# Salvataggio in JSON
with open("metadati_tif.json", "w") as f:
    json.dump(metadati, f, indent=4, default=str)

### Da sistemare e rendere più generale possibile per leggere ogni forma di tiff
## Penso che comunque l'approccio migliore sia fare una funzione per ogni macchina
